## General batching

### Sentence Transformers

In [3]:
from sentence_transformers import SentenceTransformer
from concurrent.futures import ThreadPoolExecutor

# Load your model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
model.encode("Hello, world!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


array([ 0.3059689 ,  0.79072964,  0.00980721, ...,  0.0644002 ,
       -0.45898244, -0.01831866], dtype=float32)

In [4]:
def simple_encode(text: list[str]):
    return model.encode(text)

In [5]:
with ThreadPoolExecutor(max_workers=128) as executor:
    futures = [executor.submit(simple_encode, ["Hello, world!"]) for _ in range(1000)]
    results = [future.result() for future in futures]

In [6]:
import batch

dynamic_encode = batch.dynamically(simple_encode)

with ThreadPoolExecutor(max_workers=128) as executor:
    futures = [executor.submit(dynamic_encode, ["Hello, world!"]) for _ in range(1000)]
    results = [future.result() for future in futures]

### Ofen

In [7]:
from ofen.models import TextEncoder

text_encoder = TextEncoder("mixedbread-ai/mxbai-embed-large-v1")
text_encoder.encode("Hello, world!")

/opt/homebrew/anaconda3/lib/python3.11/site-packages/ofen/models/torch/text_encoder.py:144: UserWarning: Pooling strategy is set to NONE. This may result in unexpected behavior as the output will be the full sequence of hidden states rather than a single vector per input. Consider using a different pooling strategy such as mean, max, or cls if you need a fixed-size representation.
  warnings.warn(


EncodingResult(embeddings=array([[[ 0.01729077,  0.04468535,  0.00055422, ...,  0.00363935,
         -0.0259378 , -0.00103522],
        [-0.00385489,  0.02841953,  0.0072808 , ..., -0.00641318,
         -0.00887358,  0.0145831 ],
        [ 0.02255955,  0.03204089,  0.01209312, ...,  0.00168586,
         -0.03093472,  0.01115614],
        [-0.01648994,  0.0239604 , -0.01059136, ..., -0.01044504,
         -0.04768613,  0.02494775],
        [ 0.00466856,  0.04424845,  0.00212991, ...,  0.01685553,
         -0.01741113,  0.01215234],
        [ 0.00623544,  0.03078754,  0.00834455, ...,  0.00311966,
         -0.0418628 , -0.02152618]]], dtype=float32), total_tokens=6)

In [8]:
def simple_encode(text: list[str]):
    return text_encoder.encode(text).embeddings

with ThreadPoolExecutor(max_workers=128) as executor:
    futures = [executor.submit(simple_encode, ["Hello, world!"]) for _ in range(1000)]
    results = [future.result() for future in futures]

In [10]:
import batch

dynamic_encode = batch.dynamically(simple_encode)

with ThreadPoolExecutor(max_workers=128) as executor:
    futures = [executor.submit(dynamic_encode, ["Hello, world!"]) for _ in range(1000)]
    results = [future.result() for future in futures]

## Inference batching

In [11]:
from ofen.models import TextEncoder

text_encoder = TextEncoder("mixedbread-ai/mxbai-embed-large-v1")
text_encoder.encode("Hello, world!")

EncodingResult(embeddings=array([[[ 0.01729077,  0.04468535,  0.00055422, ...,  0.00363935,
         -0.0259378 , -0.00103522],
        [-0.00385489,  0.02841953,  0.0072808 , ..., -0.00641318,
         -0.00887358,  0.0145831 ],
        [ 0.02255955,  0.03204089,  0.01209312, ...,  0.00168586,
         -0.03093472,  0.01115614],
        [-0.01648994,  0.0239604 , -0.01059136, ..., -0.01044504,
         -0.04768613,  0.02494775],
        [ 0.00466856,  0.04424845,  0.00212991, ...,  0.01685553,
         -0.01741113,  0.01215234],
        [ 0.00623544,  0.03078754,  0.00834455, ...,  0.00311966,
         -0.0418628 , -0.02152618]]], dtype=float32), total_tokens=6)

In [12]:
from batch import inference
from concurrent.futures import ThreadPoolExecutor

simple_forward = text_encoder.forward

dynamic_forward = inference.dynamically(lambda features: simple_forward(**features)["embeddings"])
text_encoder.forward = lambda **features: {"embeddings": dynamic_forward(features)}

with ThreadPoolExecutor(max_workers=128) as executor:
    futures = [executor.submit(text_encoder.encode, ["Hello, world!"]) for _ in range(1000)]
    results = [future.result() for future in futures]